In [1]:
import torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)
import warnings
warnings.filterwarnings("ignore")

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load PhoBERT emotion classifier
PHOBERT_PATH = "./phobert-emotion-final"
phobert_model = AutoModelForSequenceClassification.from_pretrained(PHOBERT_PATH)
phobert_tokenizer = AutoTokenizer.from_pretrained(PHOBERT_PATH)

id2label = phobert_model.config.id2label
print("✅ PhoBERT loaded. Labels:", list(id2label.values()))

✅ PhoBERT loaded. Labels: ['Accepting', 'Anger', 'Disappointed', 'Disgust', 'Enjoyment', 'Fear', 'Highly Negative', 'Hopeless', 'Hurt', 'Indifferent', 'Loneliness', 'Lonely', 'Neutral', 'Other', 'Sadness', 'Spam', 'Surprise']


In [3]:
# Chọn model Qwen nhỏ để chạy được trên GPU/CPU phổ thông
QWEN_MODEL_NAME = "Qwen/Qwen1.5-1.8B-Chat"

# Tải tokenizer và model
qwen_tokenizer = AutoTokenizer.from_pretrained(QWEN_MODEL_NAME, trust_remote_code=True)
qwen_model = AutoModelForCausalLM.from_pretrained(
    QWEN_MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True
)

# Tạo pipeline chat
qwen_pipe = pipeline(
    "text-generation",
    model=qwen_model,
    tokenizer=qwen_tokenizer,
    device_map="auto"
)

print("✅ Qwen loaded successfully!")

`torch_dtype` is deprecated! Use `dtype` instead!
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cpu


✅ Qwen loaded successfully!


In [4]:
def get_emotion(text):
    """Dự đoán cảm xúc bằng PhoBERT"""
    inputs = phobert_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(phobert_model.device)
    
    with torch.no_grad():
        logits = phobert_model(**inputs).logits
        pred_id = logits.argmax().item()
    return id2label[pred_id]

def generate_response(emotion, user_text, max_new_tokens=128):
    """Sinh phản hồi an ủi bằng Qwen"""
    # Tạo prompt theo cảm xúc
    prompt = f"""Bạn là một người bạn đồng cảm và nhẹ nhàng. 
Hãy phản hồi ngắn gọn (1-2 câu), chân thành và an ủi người đang cảm thấy "{emotion}".
Không cần lặp lại cảm xúc, chỉ cần thể hiện sự thấu hiểu và động viên.

Lời tâm sự: "{user_text}"

Phản hồi:"""
    
    response = qwen_pipe(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=qwen_tokenizer.eos_token_id
    )
    
    # Trích xuất text sinh ra
    full_text = response[0]['generated_text']
    reply = full_text.split("Phản hồi:")[-1].strip()
    return reply

In [5]:
test_messages = [
    "Dạo này mình cảm thấy rất cô đơn, không ai hiểu mình cả.",
    "Mình vừa bị mất việc, cảm thấy tuyệt vọng quá.",
    "Hôm nay mình rất vui vì được khen trước cả lớp!",
    "Chán nản vì học hoài mà không tiến bộ.",
    "Cảm thấy ghét bản thân vì đã làm tổn thương người khác."
]

for msg in test_messages:
    print(f"💬 Người dùng: {msg}")
    
    # Bước 1: Dự đoán cảm xúc
    emotion = get_emotion(msg)
    print(f"🧠 Cảm xúc phát hiện: {emotion}")
    
    # Bước 2: Sinh phản hồi
    reply = generate_response(emotion, msg)
    print(f"💬 Qwen: {reply}")
    print("-" * 80)

💬 Người dùng: Dạo này mình cảm thấy rất cô đơn, không ai hiểu mình cả.
🧠 Cảm xúc phát hiện: Sadness
💬 Qwen: "Xin lỗi, tôi biết rằng cảm xúc như vậy thường có nhiều tác động tích cực đến sức khỏe và hạnh phúc của chúng ta. Tôi có thể giúp gì cho bạn trong thời gian này? Bạn có thể nói về những điều mà bạn cảm thấy cô đơn nhất và tôi sẽ cố gắng tìm kiếm giải pháp để giúp bạn trở nên tự do hơn và tiếp tục sống tốt đẹp hơn."
--------------------------------------------------------------------------------
💬 Người dùng: Mình vừa bị mất việc, cảm thấy tuyệt vọng quá.
🧠 Cảm xúc phát hiện: Sadness
💬 Qwen: **Sự thách thức:** Tôi cảm thấy rất khó khăn vì tôi đã làm việc cho công ty này hàng năm, và tôi không thể tìm được một vị trí mới có thể giúp tôi trở về trong tương lai. Tôi luôn nghĩ rằng mình sẽ tiếp tục làm việc tốt hơn trong tương lai và tôi muốn tiếp tục được hỗ trợ của công ty để giúp tôi đối phó với những thử thách này.
**Tài liệu:** Tôi đã dành nhiều thời gian và công sức để giáo dục 

In [ ]:
def chat():
    print("💬 Xin chào! Bạn có thể chia sẻ bất kỳ điều gì — mình luôn lắng nghe.")
    while True:
        user_input = input("\nBạn: ").strip()
        if user_input.lower() in ['quit', 'exit', 'thoát']:
            print("Tạm biệt! ❤️")
            break
        if not user_input:
            continue
            
        emotion = get_emotion(user_input)
        reply = generate_response(emotion, user_input)
        print(f"\n🧠 Cảm xúc: {emotion}")
        print(f"💬 Qwen: {reply}")